# 행정안정부 도로명주소 API

## 건물명으로부터 도로명주소 구하기

In [3]:
import pandas as pd
import numpy as np

import requests
from urllib.parse import urlparse, quote

In [4]:
key_fd = open('roadapikey.txt', mode='r')
roadapi_key = key_fd.read(100)
key_fd.close()

In [5]:
road_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
confmKey = roadapi_key
currentPage = 1
countPerPage = 5
keyword = quote('강서구청')
resultType = 'json'

request_url = f"{road_url}?confmKey={roadapi_key}&currentPage={currentPage}&countPerPage={countPerPage}&keyword={keyword}&resultType=json"

urlparse(request_url).geturl()

'https://www.juso.go.kr/addrlink/addrLinkApi.do?confmKey=U01TX0FVVEgyMDIwMTExOTEwMjMzNTExMDQzNzg=&currentPage=1&countPerPage=5&keyword=강서구청&resultType=json'

In [6]:
quote('강서구청')

'%EA%B0%95%EC%84%9C%EA%B5%AC%EC%B2%AD'

In [ ]:
request_url = f"{road_url}?confmKey={roadapi_key}&currentPage={currentPage}&countPerPage={countPerPage}&keyword={keyword}&resultType=json"

In [4]:
results = requests.get(urlparse(request_url).geturl()).json()
addr_list = results['results']['juso']
addr_list[0]

{'detBdNmList': '',
 'engAddr': '302, Hwagok-ro, Gangseo-gu, Seoul',
 'rn': '화곡로',
 'emdNm': '화곡동',
 'zipNo': '07658',
 'roadAddrPart2': ' (화곡동)',
 'emdNo': '02',
 'sggNm': '강서구',
 'jibunAddr': '서울특별시 강서구 화곡동 980-16 강서구청',
 'siNm': '서울특별시',
 'roadAddrPart1': '서울특별시 강서구 화곡로 302',
 'bdNm': '강서구청',
 'admCd': '1150010300',
 'udrtYn': '0',
 'lnbrMnnm': '980',
 'roadAddr': '서울특별시 강서구 화곡로 302 (화곡동)',
 'lnbrSlno': '16',
 'buldMnnm': '302',
 'bdKdcd': '0',
 'liNm': '',
 'rnMgtSn': '115003005069',
 'mtYn': '0',
 'bdMgtSn': '1150010300109800016015086',
 'buldSlno': '0'}

In [5]:
for addr in addr_list:
    print(addr['bdNm'], addr['roadAddr'])

강서구청 서울특별시 강서구 화곡로 302 (화곡동)
강서구청 서울특별시 강서구 화곡로44나길 72 (화곡동)
강서구청 부산광역시 강서구 낙동북로 477 (대저1동)
강서구청청소차고 서울특별시 강서구 남부순환로 208 (외발산동)
강서구청가양동별관 서울특별시 강서구 양천로59길 38 (가양동)


## DataFrame

In [6]:
def getAPI(**kwargs):
    confmKey = roadapi_key
    baseUrl = "https://www.juso.go.kr/addrlink/addrLinkApi.do?confmKey="
    params = []

    for k, v in kwargs.items():
        params.append("{}={}".format(k, v))        

    param = '&' + '&'.join(params)
    query = baseUrl + confmKey + param
    
    return requests.get(urlparse(query).geturl()).json()

In [7]:
data1 = getAPI(currentPage=1, countPerPage=5, keyword='강서구청', resultType='json')
data2 = getAPI(currentPage=1, countPerPage=5, keyword='양천구청', resultType='json')
data3 = getAPI(currentPage=1, countPerPage=5, keyword='부천시청', resultType='json')
data4 = getAPI(currentPage=1, countPerPage=5, keyword='영등포구청', resultType='json')
data5 = getAPI(currentPage=1, countPerPage=5, keyword='김포시청', resultType='json')

In [8]:
juso = [
    data1['results']['juso'][0],
    data2['results']['juso'][0],
    data3['results']['juso'][0],
    data4['results']['juso'][0],
    data5['results']['juso'][0]
]
df = pd.DataFrame(juso)
df

,detBdNmList,engAddr,rn,emdNm,zipNo,roadAddrPart2,emdNo,sggNm,jibunAddr,siNm,...,lnbrMnnm,roadAddr,lnbrSlno,buldMnnm,bdKdcd,liNm,rnMgtSn,mtYn,bdMgtSn,buldSlno
0,,"302, Hwagok-ro, Gangseo-gu, Seoul",화곡로,화곡동,07658,(화곡동),02,강서구,서울특별시 강서구 화곡동 980-16 강서구청,서울특별시,...,980,서울특별시 강서구 화곡로 302 (화곡동),16,302,0,,115003005069,0,1150010300109800016015086,0
1,,"105, Mokdongdong-ro, Yangcheon-gu, Seoul",목동동로,신정동,08095,(신정동),01,양천구,서울특별시 양천구 신정동 321-4 양천구청,서울특별시,...,321,서울특별시 양천구 목동동로 105 (신정동),4,105,0,,114703114001,0,1147010100103210004000689,0
2,,"210-2, Gilju-ro, Bucheon-si, Gyeonggi-do",길주로,중동,14547,(중동),02,부천시,경기도 부천시 중동 1156 부천시청 어린이집,경기도,...,1156,경기도 부천시 길주로 210-2 (중동),0,210,0,,411903000065,0,4119010800111560000000001,2
3,,"123, Dangsan-ro, Yeongdeungpo-gu, Seoul",당산로,당산동3가,07260,(당산동3가),03,영등포구,서울특별시 영등포구 당산동3가 385-1 영등포구청,서울특별시,...,385,서울특별시 영등포구 당산로 123 (당산동3가),1,123,0,,115603118004,0,1156011300103850001029173,0
4,"민방위교육장,민원봉사실,본관동, 의회동,부대시설,신별관동,정보관동,주차요금정산서,팔각정","1, Saujung-ro, Gimpo-si, Gyeonggi-do",사우중로,사우동,10109,(사우동),02,김포시,경기도 김포시 사우동 263-1 김포시청,경기도,...,263,경기도 김포시 사우중로 1 (사우동),1,1,0,,415703209060,0,4157010600102630001022084,0


In [12]:
df.to_csv('address.csv')